# Deep Q-Learning 

Reference:
+ [Deep Q-Learning: A Neural Network Approach to Learning Without Experience Replay](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf)
  

## I. Problem Formulation

From the [Problem Formulation](ProblemFormulation_Notation.ipynb), the objective of RL is to maximize the expected returns:
    $$\pi^*= \argmax_\pi J(\pi) \quad \text{where} \quad J(\pi)= \mathbb{E}_{\tau \sim \pi} [R(\tau)]  \quad\quad (1)$$
where $R(\tau) = \sum_{t=0}^T \gamma^t r_t$ is the accumulated returns of a trajectory $\tau=(s_0,a_0,s_1,a_1,...)$. 

For a given initial state $s_0$, or state-action pair $(s_0,a_0)$, we can define the Value-Function and Q-Function:
+ *Value function*: $V^{\pi}(s) = \mathbb{E}_{\tau \sim \pi} [R(\tau) | s_0=s]$ 
+ *Action-Value function (a.k.a Quality or Q-Function)*: $Q^{\pi}(s,a) = \mathbb{E}_{\tau \sim \pi} [R(\tau) | s_0=s, a_0=a]$ 

Off-Policy methods do not aim to find the policy $\pi$ directly, but instead to build the function $Q(s,a)$. **The intuition is very simple:** if we know $Q(s,a)$, we can simply pick the optimal action $a=\argmax Q(\cdot|s)$ for any state $s$. For example, for an environment with state space $s \in R^n$ and action space $a \in R^m$,
+ **Discrete Action:** If the action is discrete, $a_k=\{0,1\},k=1:m$, we can build neural network $Q_\phi(\cdot|s): R^n \rightarrow R^m$ parameterized by $\{\phi\}$, and pick the action $a_k$ that has highest $Q$, e.g. $Q(a_k) \geq Q(a_m)$ for $m \neq k$.
+ **Continuous Action:** If the action is continuous $a_k \in R,k=1:m$, we can build neural network $Q_\phi(a,s): R^{n+m} \rightarrow R$, and select $a = \argmax Q_\phi(a,s)$ , e.g. by solving $\nabla_a Q(a,s)=0$.  

Therefore, the problem reduces to how to build the function $Q(a,s)$.


## II. Solution 
### 1.Bellman Equation:
From the definition of accumulated returns $R_t=\sum_{t=0}^\infty \gamma^t r_t$, where $r_t=r(s_t,a_t,s_{t+1})$ is the reward received after acting action $a_t$ at the state $s_t$ and arrive at the new state $s_{t+1}$, we can write it in recursive form:
    $$ R_t = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + \cdots + \gamma^{T-1} r_{T}  = r_t + \gamma R_{t+1} \quad \quad (2)$$
**Hence, from the definition of Q-function**, we can write it in the recursive form:  
    $$ \boxed{ Q^{\pi}(s_t,a_t)  = \mathbb{E}_{\tau \sim \pi} [R(\tau) | s_0=s_t, a_0=a_t] = \sum_{s_{t+1}} P(s_{t+1}|s_t,a_t) \left[ r(s_t,a_t,s_{t+1}) + \gamma \sum_{a_{t+1}} \pi(a_{t+1}|s_{t+1})Q^{\pi}(s_{t+1},a_{t+1}) \right] }\quad \quad (3)$$
where:
+ The first term $\sum_{s_{t+1}} P(s_{t+1}|s_t,a_t) [\cdots ]$ indicates that we can arrive any state $s_{t+1}$ after taking action $a_t$ from the state $s_t$ due to the stochastic of environment, and receive the corresponding reward $r(s_t,a_t,s_{t+1})$.
+ The second term $\sum_{a_{t+1}} \pi(a_{t+1}|s_{t+1}) [\cdots ]$ indicates that we can take any action $a_{t+1}$ due to the stochastic of the policy.
+ The last term $Q^{\pi}(s_{t+1},a_{t+1})$ indicates that we need to repeatedly write the first and second sum for all later actions and states. 

**Similarly, from the definition of Value-function**, we can write it in the recursive form:
    $$ \boxed{ V^{\pi}(s_t)  = \mathbb{E}_{\tau \sim \pi} [R(\tau) | s_0=s_t] = \sum_{a_t} \pi(a_t|s_t) \left[ \sum_{s_{t+1}}  P(s_{t+1}|s_t,a_t) [r(s_t,a_t,s_{t+1}) + \gamma V^{\pi}(s_{t+1})] \right] } \quad \quad (4) $$
where:
+ The first term $\sum_{a_t} \pi(a_t|s_t) [\cdots ]$ indicates that we can take any action $a_t$ from the state $s_t$ due to the stochastic of the policy.
+ The second term $\sum_{s_{t+1}}  P(s_{t+1}|s_t,a_t) [\cdots ]$ indicates that we can arrive any state $s_{t+1}$ after taking action $a_t$ from the state $s_t$ due to the stochastic of environment. After that, we get the first reward $r(s_t,a_t,s_{t+1})$.
+ The last term $V^{\pi}(s_{t+1})$ indicates that we need to repeatedly write the first and second sum for all later actions and states. 

**(3) and (4) are called Bellman equation,** and there is the connection between V-Function and Q-Function by their definition:
    $$ V^{\pi}(s_t) = \mathbb{E}_{\tau \sim \pi} [R(\tau) | s_0=s_t] = \sum_{a_t} \pi(a_t|s_t) Q^{\pi}(s_t,a_t) \quad \quad (5)$$
Substituting (5) into (3), we get (6) respectively:
    $$ Q^{\pi}(s_t,a_t) = \sum_{s_{t+1}} P(s_{t+1}|s_t,a_t) \left[ r(s_t,a_t,s_{t+1}) + \gamma V^{\pi}(s_{t+1}) \right] \quad \quad (6)$$

### 2. Optimal Policy by Q-Learning:
By definition **the optimal policy is deterministic and greedy**: if there is an action with a maximal Q-value for the optimal policy, it should be systematically taken. For the optimal policy $\pi^*$, the Bellman equations become:
\begin{align*}
    V^*(s_t) &= \max_{a} \sum_{s_{t+1}}  P(s_{t+1}|s_t,a_t) [r_t + \gamma  V^*(s_{t+1})] \quad \quad (7)\\
    Q^*(s_t,a_t) &= \sum_{s_{t+1}}  P(s_{t+1}|s_t,a_t) [r_t + \gamma \max_{a_{t+1}}  Q^*(s_{t+1})] = \mathbb{E}_{s_{t+1} \sim P(s_{t+1}|s_t,a_t)} [r_t + \gamma \max_{a_{t+1}}  Q^*(s_{t+1})] \quad \quad (8)\\
\end{align*}

### 3. How to approximate Q-Function:
From the above analysis, especially the Equ (8), if we ignore the stochastic of the environment, we can get the following property:
    $$Q^*(s_t,a_t) = r_t + \gamma \max_{a_{t+1}}  Q^*(s_{t+1})$$
Therefore, we can empose this property into the loss function to update the Q-Network. Concretely, the loss function is defined as:
    $$L(Q_\phi) = \frac{1}{2}\mathbb{E}_{s_t,a_t,s_{t+1},a_{t+1}}(Q_{target} - Q_\phi(s_t,a_t))^2 \quad \text{where} \quad Q_{target}= r_t + \gamma \max_{a_{t+1}}  Q_\phi(s_{t+1}) \quad \quad (9)$$
Then we can use SGD to update the Q-network: $\phi \leftarrow \phi - \eta \nabla_\phi L(Q_\phi) $.